In [23]:
import numpy as np
import simplex
from copy import copy

In [24]:
def is_integer(x, eps=10 ** (-10)):
    return abs(x - round(x)) <= eps

In [25]:
def gomori(A, b, c):
    A_in = np.copy(A)
    b_in = copy(b)
    c_in = np.copy(c)
    initial_len = None
    
    while True:
        # Get real solution using simplex method
        
        real_solution, basis_indices = simplex.solve(A_in, b_in, c_in)

        if real_solution == None:
            return "No Integer Solution"
        
        # Check if all values of solution are integers
        
        if all(is_integer(val) for val in real_solution):
            if (initial_len != None):
                real_solution = real_solution[:initial_len]
            return real_solution
        else:
            if (initial_len == None):
                initial_len = len(real_solution)
                
            k, x_k = None, -np.inf 
            
            for index, item in enumerate(real_solution):
                if not is_integer(item) and index in basis_indices and np.modf(item)[0] > x_k:
                    k, x_k = index, item
            
            k = basis_indices.index(k)
        
        # Fill A_b and A_0 matrix
        m = len(basis_indices)
        n = len(b_in)
        J = [i for i in range(len(c_in))]
        non_basis_indices = [i for i in set(J).difference(basis_indices)]
        m_0 = len(non_basis_indices)
        
        A_b = np.zeros((n, m))
        A_0 = np.zeros((n, m_0))
        
        for i, j in enumerate(basis_indices):
            A_b[:, i] = A_in[:, j]
            
        for i, j in enumerate(non_basis_indices):
            A_0[:, i] = A_in[:, j]
         
        # Invert A_b matrix
        A_b = np.linalg.inv(A_b)
        
        # Multiply A_b and A_0
        M = A_b.dot(A_0)
        
        # Get vector l, fill new var
        l = M[k]
        
        c_in = np.append(c_in, 0)
        row = np.array([])
        shiish = 0
        
        for index in J:
            if index in non_basis_indices:
                row = np.append(row, l[shiish])
                shiish += 1
            else:
                row = np.append(row, 0)
        
        A_in = np.append(A_in, [row], axis=0)
        
        len_A = len(A_in)
        column = np.zeros(len_A)
        column[len_A - 1] = -1
        
        new_var_col = [[i] for i in column]
        A_in = np.append(A_in, new_var_col, axis=1)
        b_in.append(np.modf(x_k)[0])

In [26]:
A = np.array([
    [-4, 6, 1, 0],
    [1, 1, 0, 1]
])
b = [9, 4]
c = np.array([-1, 2, 0, 0])

In [27]:
print('Result:')
sol = gomori(A, b, c)

if isinstance(sol, str):
    print(sol)
else:
    formatted_sol = ', '.join(map(str, map(int, sol)))
    print('Solution x: ', end='')
    print(f'({formatted_sol})')

Result:
Solution x: (1, 2, 0, 1)
